In [110]:
import numpy as np

class Point:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)
    
    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y)
    
    def __eq__(self, __o: object) -> bool:
        return self.x == __o.x and self.y == __o.y
    
    def __abs__(self):
        return (self.x ** 2 + self.y ** 2) ** 0.5
    
    def __neg__(self):
        return Point(-self.x, -self.y)
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    @property
    def x(self):
        return self.__x
    @x.setter
    def x(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("x must be a number")
        self.__x = float(value)
        
    @property
    def y(self):
        return self.__y
    @y.setter
    def y(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("y must be a number")
        self.__y = float(value)
        
    @property
    def tuple(self):
        return (self.x, self.y)
    @tuple.setter
    def tuple(self, value):
        if not isinstance(value, tuple):
            raise TypeError("tuple must be a tuple")
        if len(value) != 2:
            raise ValueError("tuple must be a tuple of length 2")
        self.x = value[0]
        self.y = value[1]
    @property
    def list(self):
        return [self.x, self.y]
    @list.setter
    def list(self, value):
        if not isinstance(value, list):
            raise TypeError("list must be a list")
        if len(value) != 2:
            raise ValueError("list must be a list of length 2")
        self.x = value[0]
        self.y = value[1]

    
    @classmethod
    def from_tuple(cls, point):
        return cls(point[0], point[1])
    
    @staticmethod
    def distance(p1, p2):
        return ((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2) ** 0.5
    @staticmethod
    def midpoint(p1, p2):
        return Point((p1.x + p2.x) / 2, (p1.y + p2.y) / 2)
    @staticmethod
    def slope(p1, p2):
        if p2.x - p1.x == 0:
            return np.inf
        return (p2.y - p1.y) / (p2.x - p1.x)
    @staticmethod
    def intercept(p1, p2):
        if p2.x - p1.x == 0:
            return np.nan
        return p1.y - Point.slope(p1, p2) * p1.x
    @staticmethod
    def equation(p1, p2):
        if p2.x - p1.x == 0:
            return f"x = {p1.x}"
        return f"y = {Point.slope(p1, p2)}x + {Point.intercept(p1, p2)}"
    @staticmethod
    def equation_slope_intercept(slope, intercept):
        if slope == np.inf:
            return f"x = {intercept}"
        return f"y = {slope}x + {intercept}"
    @staticmethod
    def is_parallel(p1, p2, p3, p4):
        return Point.slope(p1, p2) == Point.slope(p3, p4)
    @staticmethod
    def is_perpendicular(p1, p2, p3, p4):
        return Point.slope(p1, p2) * Point.slope(p3, p4) == -1

    @staticmethod
    def intersection(p1, p2, p3, p4):
        slope1 = Point.slope(p1, p2)
        slope2 = Point.slope(p3, p4)
        intercept1 = Point.intercept(p1, p2)
        intercept2 = Point.intercept(p3, p4)
        
        if slope1 == slope2:
            return None
        
        if slope1 == np.inf:
            x = p1.x
            y = slope2 * x + intercept2
        elif slope2 == np.inf:
            x = p3.x
            y = slope1 * x + intercept1
        else:
            x = (intercept2 - intercept1) / (slope1 - slope2)
            y = slope1 * x + intercept1
        return Point(x, y)
    
    @staticmethod
    def is_collinear(p1, p2, p3):
        return Point.is_parallel(p1, p2, p2, p3)
    
    @staticmethod
    def area(p1, p2, p3):
        return abs((p1.x * (p2.y - p3.y) + p2.x * (p3.y - p1.y) + p3.x * (p1.y - p2.y)) / 2)
    
    @staticmethod
    def is_point_inside_triangle(p, p1, p2, p3):
        A = Point.area(p1, p2, p3)
        A1 = Point.area(p, p2, p3)
        A2 = Point.area(p1, p, p3)
        A3 = Point.area(p1, p2, p)
        return A == A1 + A2
    
    @staticmethod
    def is_point_inside_rectangle(p, p1, p2, p3, p4):
        return Point.is_point_inside_triangle(p, p1, p2, p3) or Point.is_point_inside_triangle(p, p1, p3, p4)
    
    @staticmethod
    def is_point_inside_circle(p, center, radius):
        return Point.distance(p, center) <= radius
    
    @staticmethod
    def is_point_inside_polygon(p, *points):
        if len(points) < 3:
            return False
        
        A = 0
        for i in range(len(points)):
            A += Point.area(p, points[i], points[(i + 1) % len(points)])
        
        A1 = 0
        for i in range(len(points)):
            A1 += Point.area(points[i], points[(i + 1) % len(points)], points[(i + 2) % len(points)])
        
        return A == A1
    @staticmethod
    def points_outside_polygon(*points):
        if len(points) < 3:
            return []
        
        outside = []
        for i in range(len(points)):
            if not Point.is_point_inside_triangle(points[i], points[(i + 1) % len(points)], points[(i + 2) % len(points)], points[(i + 3) % len(points)]):
                outside.append(points[i])
        return outside
    
    @staticmethod
    def outside_area(*points):
        if len(points) < 3:
            return 0
        outside = Point.points_outside_polygon(*points)
        A = 0
        for i in range(len(outside)):
            A += Point.area(outside[i], outside[(i + 1) % len(outside)], outside[(i + 2) % len(outside)])
        return A
    
    @staticmethod
    def closest_point(p, *points):
        if len(points) == 0:
            return None
        
        closest = points[0]
        for point in points:
            if Point.distance(p, point) < Point.distance(p, closest):
                closest = point
        return closest
    @staticmethod
    def farthest_point(p, *points):
        if len(points) == 0:
            return None
        
        farthest = points[0]
        for point in points:
            if Point.distance(p, point) > Point.distance(p, farthest):
                farthest = point
        return farthest
    @staticmethod
    def closest_points_in_order(p, *points):
        if len(points) == 0:
            return None
        
        points = list(points)
        points.sort(key=lambda point: Point.distance(p, point))
        return points

    

In [158]:
class Edge:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2     
    def __repr__(self):
        return f"edge({self.p1}, {self.p2})"
    def __str__(self):
        return f"edge({self.p1}, {self.p2})"
    def __eq__(self, other):
        return self.p1 == other.p1 and self.p2 == other.p2
    def __abs__(self):
        return Point.distance(self.p1, self.p2)
    def __contains__(self, point):
        return Point.is_collinear(self.p1, self.p2, point)
    def __add__(self, other):
        return Edge(self.p1 + other.p1, self.p2 + other.p2)
    def __sub__(self, other):
        return Edge(self.p1 - other.p1, self.p2 - other.p2)
    def __hash__(self):
        return hash((self.p1, self.p2))
    
    @property
    def slope(self):
        return Point.slope(self.p1, self.p2)
    @property
    def intercept(self):
        return Point.intercept(self.p1, self.p2)
    @property
    def equation(self):
        return Point.equation(self.p1, self.p2)
    @property
    def equation_slope_intercept(self):
        return Point.equation_slope_intercept(self.slope, self.intercept)
    
    @classmethod
    def from_points(cls, *points):
        return Edge(points[0], points[1])
    
    @staticmethod
    def is_parallel(self, other):
        return Point.is_parallel(self.p1, self.p2, other.p1, other.p2)
    @staticmethod
    def is_perpendicular(self, other):
        return Point.is_perpendicular(self.p1, self.p2, other.p1, other.p2)
    @staticmethod
    def intersection(self, other):
        return Point.intersection(self.p1, self.p2, other.p1, other.p2)
    @staticmethod
    def is_collinear(self, other):
        return Point.is_collinear(self.p1, self.p2, other.p1)
    @staticmethod
    def distance_from_point(self, point):
        return min(Point.distance(point, self.p1), Point.distance(point, self.p2))
    @staticmethod
    def closest_vertex(self, point):
        if Point.distance(point, self.p1) < Point.distance(point, self.p2):
            return self.p1
        return self.p2
    @staticmethod
    def farthest_vertex(self, point):
        if Point.distance(point, self.p1) > Point.distance(point, self.p2):
            return self.p1
        return self.p2
    @staticmethod
    def closest_edge(self, *edges):
        if len(edges) == 0:
            return None
        
        closest = edges[0]
        for edge in edges:
            if self.distance_from_point(edge) < self.distance_from_point(closest):
                closest = edge
        return closest
    @staticmethod
    def closest_edges_in_order(self, *edges):
        if len(edges) == 0:
            return None
        
        edges = list(edges)
        edges.sort(key=lambda edge: self.distance_from_point(edge))
        return edges
    

In [ ]:
class Graph:
    def __init__(self, *points, **connections):
        self.points = list(points)
        self.edges = edges
        
    def __repr__(self) -> str:
        return f"graph({self.points}, {self.edges})"
    def __str__(self) -> str:
        return f"graph({self.points}, {self.edges})"
    def __contains__(self, point):
        return point in self.points
    def __iter__(self):
        return iter(self.points)
    def __len__(self):
        return len(self.points)
    def __getitem__(self, index):
        return self.points[index]
    def __abs__(self):
        return sum(abs(edge) for edge in self.edges.values())
    def __add__(self, other):
        return Graph(*(self.points + other.points), **{**self.edges, **other.edges})
    def __sub__(self, other):
        return Graph(*(self.points - other.points), **{**self.edges, **other.edges})
    
    @property
    def vertices(self):
        return self.points
    @property
    def edges(self):
        return self.edges
    @property
    def area(self):
        return Point.outside_area(*self.points)
    

In [156]:
class Network:
    def __init__(self, nodes: dict):
        self.vertices = nodes.keys()
        self.edges = set()
        for node,edge_list in nodes.items():
            for edge in edge_list:
                self.edges.add(Edge(node, edge))
        
    
    def __repr__(self) -> str:
        return f"network({self.vertices}, {self.edges})"
    def __str__(self) -> str:
        return f"network({self.vertices}, {self.edges})"
    def __contains__(self, point):
        return point in self.vertices
    def __iter__(self):
        return iter(self.vertices)
    def __len__(self):
        return len(self.vertices)
    def __getitem__(self, index):
        return self.vertices[index]
    


In [159]:
network = Network({Point(0,1) : [Point(1,1), Point(0,2)], Point(1,1) : [Point(0,1), Point(1,2)], Point(0,2) : [Point(0,1), Point(1,2)], Point(1,2) : [Point(1,1), Point(0,2)]})

In [162]:
network.edges

{edge((0.0, 1.0), (0.0, 2.0)),
 edge((0.0, 1.0), (1.0, 1.0)),
 edge((0.0, 2.0), (0.0, 1.0)),
 edge((0.0, 2.0), (1.0, 2.0)),
 edge((1.0, 1.0), (0.0, 1.0)),
 edge((1.0, 1.0), (1.0, 2.0)),
 edge((1.0, 2.0), (0.0, 2.0)),
 edge((1.0, 2.0), (1.0, 1.0))}

In [153]:
def fn(*args, **kwargs):
    for i in args:
        print(i)
    for i in kwargs:
        print(i)
kwdic = {2: 1, 3: 2}

In [155]:
fn(1, 2, 3, **kwdic)

TypeError: keywords must be strings